# Default networks

Docker comes with three nets by default.

In [1]:
!docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
1726f609501a   bridge    bridge    local
80c3e6772c8c   host      host      local
45bb103d970f   none      null      local


## Network `none`

By connecting the container to this network, it is completely blocked from any interactions through the network.

In the following cell, there is a simple bash script that allows you to compare the behavior of the container to make it make a fucking bug?

### From host to container

In the following cell, there is a simple bash script that allows you to compare the behavior of a container connected to the `bridge` network and a container connected to the `none` network.

At the beginning, a container is started using the `bridge` network. When the command `curl localhost:80` is executed, a basic nginx response is received. 

Then, the network for the container is changed to `none`. As a consequence, when the command `curl localhost:80` is executed again, there is no response. This lack of response is a consequence of using the `none` network for the container.

Overall, this script demonstrates the impact of network selection on the container's ability to communicate and receive responses.

In [2]:
%%bash
docker run --rm -itd\
    -p 80:80 \
    --name network_example\
    nginx
sleep 2 &> /dev/null

echo "=====DEFAULT NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl to nginx-----"
curl -s localhost:80 | head -n 10

docker network disconnect bridge network_example
docker network connect none network_example

echo
echo
echo "=====none NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl to nginx-----"
curl -s localhost:80 | head -n 10

docker stop network_example &> /dev/null

4a093f76ed7b8b10c4ef46873ada94a5b49fce7266b81588031693ba72dd1a42
=====DEFAULT NETWORK=====
-----network name-----
bridge 
-----curl to nginx-----
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>


=====none NETWORK=====
-----network name-----
none 
-----curl to nginx-----


### From container to internet

Another feature of the `none` network is that it does not have access to the internet. for instance, you can use the `curl` command to fetch data from any website while the docker container is connected to the `bridge` network. however, if you switch the network to `none`, the `curl` command won't work.

The script demonstrates this scenario. initially, the container is connected to the `bridge` network, and executing the `curl example.com` command within the container successfully fetches the webpage. but after switching the container to the `none` network, the same `curl` command does not produce any output.

In [4]:
%%bash
docker run --rm -itd\
    --name network_example\
    nginx
sleep 2 &> /dev/null

echo "=====DEFAULT NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl from bridge-----"
docker exec network_example curl -s http://example.com/ | head -n 10

docker network disconnect bridge network_example
docker network connect none network_example

echo
echo
echo "=====none NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl from none-----"
docker exec network_example curl -s http://example.com/ | head -n 10

docker stop network_example &> /dev/null

7c305926adedbbe6064780448036ac5ff840e859cddfaec9fd0ffa4277b2090c
=====DEFAULT NETWORK=====
-----network name-----
bridge 
-----curl from bridge-----
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {


=====none NETWORK=====
-----network name-----
none 
-----curl from none-----
